# CSV to Label image
---

csv 파일(summary)와 생성 대상 목록을 읽어 지정된 위치에 라벨을 생성한다.

In [49]:
import os 
import pandas as pd
import cv2
import numpy as np
import re

In [118]:
print(SUMMARY_PATH)
print(LIST_PATH)
print(SAVE_PATH)

/home/aiffel-dj1/workspace/Hackerton/RSI_OP_NIA_AIHUB/merged/Buildings.csv
/home/aiffel-dj1/workspace/Hackerton/RSI_OP_NIA_AIHUB/B_train_list.txt
/home/aiffel-dj1/workspace/Hackerton/RSI_OP_NIA_AIHUB/buildings/training/label


In [82]:
SUMMARY_PATH = os.path.join(os.getenv('HOME'), 'workspace/Hackerton/RSI_OP_NIA_AIHUB/merged/Buildings.csv')
LIST_PATH = os.path.join(os.getenv('HOME'), 'workspace/Hackerton/RSI_OP_NIA_AIHUB/B_train_list.txt')
SAVE_PATH = os.path.join(os.getenv('HOME'), 'workspace/Hackerton/RSI_OP_NIA_AIHUB/buildings/training/label')
IMG_WIDTH = 1024
IMG_HEIGHT = 1024
CLASSES = {
    'building' : 0,
    'road' : 1
    
}

_color_map = [
    (165, 42, 42),
    (0, 192, 0),
    (255,255,255)
]

In [9]:
df = pd.read_csv(SUMMARY_PATH)
df.head(10)

,image_id,type,coordinates_pix,coordinates_geo
0,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((0 149, 0 91, 36 66, 43 74, 49 70, 77...","POLYGON ((-104.8997128936 39.72242979929785, -..."
1,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((126 110, 176 176, 203 155, 159 95, 1...",POLYGON ((-104.8986826105615 39.72267606503399...
2,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((117 72, 137 98, 182 66, 162 39, 117 ...",POLYGON ((-104.8987562022071 39.72291601626406...
3,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((185 6, 257 105, 286 83, 222 0, 191 0...",POLYGON ((-104.8982001764403 39.72333277366367...
4,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((189 179, 207 203, 253 169, 235 145, ...",POLYGON ((-104.8981674690423 39.72224036411621...
5,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((219 225, 229 240, 269 212, 261 202, ...",POLYGON ((-104.8979221635569 39.72194989683769...
6,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((235 243, 251 266, 269 251, 269 245, ...",POLYGON ((-104.8977913339647 39.72183623572871...
7,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((0 344, 5 350, 15 345, 32 371, 25 374...","POLYGON ((-104.8997128936 39.72119847061719, -..."
8,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((0 441, 6 435, 22 456, 0 478, 0 441))","POLYGON ((-104.8997128936 39.72058596352989, -..."
9,OBJ00875_PS3_K3_NIA0089,building,"POLYGON ((10 431, 26 450, 80 411, 72 400, 25 4...",POLYGON ((-104.8996311251049 39.72064910859043...


In [17]:
target_list = []

try:
    with open(LIST_PATH, 'r') as f:
        target_list = f.readlines()
except IOError:
    print("Error: File does not appear to exist.")


target_list = [file.split('.')[0] for file in target_list]
target_list[:5]

['BLD00001_PS3_K3A_NIA0276',
 'BLD00002_PS3_K3A_NIA0276',
 'BLD00004_PS3_K3A_NIA0276',
 'BLD00006_PS3_K3A_NIA0276',
 'BLD00008_PS3_K3A_NIA0276']

In [110]:
def wkt_to_list(s):
    nums = re.findall(r'\d+(?:\.\d*)?', s)
    nums = [round(float(p)) for p in nums]
    coords = zip(*[iter(nums)] * 2)
    return np.array(list(coords), np.int32)

In [114]:
for image_id in target_list[:1]:
    df_sub = df[df['image_id'] == image_id]
    polygons = df_sub['coordinates_pix'].array
    _type = df_sub['type'].array
    
    label = np.full((IMG_WIDTH, IMG_HEIGHT, 3), 255, np.uint8)
    for polygon, _class in zip(polygons, _type):
        coords = wkt_to_list(polygon)
        label = cv2.fillPoly(bg, [coords], color=(165, 42, 42))
        label = cv2.polylines(bg, [coords], True, color=(255,255,255), thickness=1)
        
        cv2.imwrite(f'{SAVE_PATH}/{image_id}.png', cv2.cvtColor(label, cv2.COLOR_BGR2RGB))